<a href="https://colab.research.google.com/github/SL-LAIDLAW/projet-outilBioinfo/blob/dev/Projet_Outils_Bioinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet de HMSN204
_Sean LAIDLAW_, _Mathieu Blaison_

## Traitements Biopython de SEX1

### Obtention des sequences sauvages et mutés, du NCBI

#### Obtention du sequence sauvage

In [1]:
!pip install biopython
!pip install pandas
from Bio import SeqIO 
from Bio import Entrez

/usr/local/lib/python2.7/site-packages/pip/_vendor/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
    100% |████████████████████████████████| 2.3MB 1.3MB/s ta 0:00:011
/usr/local/lib/python2.7/site-packages/pip/_vendor/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


À partir de TAIR, on a obtenu l'identifiant correspondant au *full length cDNA* de la sequence genomique de SEX1. Un requete envers la base de donnée nucleotide du NCBI nous a fourni le fasta de cette sequence.

In [2]:
Entrez.email = "sean.laidlaw@etu.umontpellier.fr"
seqgb  = Entrez.efetch (db="nucleotide",id='NM_001331926.1', rettype='fasta')
seqselect = SeqIO.parse(seqgb, "fasta")

sex1_sauvage = []
for sequence in seqselect:
    sex1_sauvage = sequence.seq
sex1_sauvage
print(sex1_sauvage)

TATCTTCTGCAATTCGTGGTTTGTGAACACAAGAGAAAAAAAAAAAAAAACATAAAAAAGAAAGAAAAATTAAAAAAAGCTAAAATCAACGGTGAGAAGATTCTGTATATAATAAGAAAATATCACCCAAATAAAAAAAAAAGGTGGAGTCCACAGATACCGTCCATCAACGTATCACTGGGCTCTCTTTCTCCGTGCCGTTCCTCCTACGTGCTCGAATAACGATGTGCCACGTGGTGGAATCGAAGGATAAAGTGTCGTTTTTTACTTCGATTACACGTGAGTGTATGTATGTGTGAGTGAGTGAAGAAGAACAAAGAGCGAAGAAACCCCCCAGCACTCACTTCATCCATAAGATCATTCAAATTCTGGGATCTTCTCTCTTTCTCTCATTGTATTCAGTTTCTTCGTCTCGGTTCGATCATCAGATCCGGCAAGTTTGATTCAGTGACAAGAAAGCATGAGTAACTCTGTAGTGCATAACTTACTTAACCGGGGTTTGATTCGTCCTCTTAACTTTGAACATCAAAACAAGCTCAACTCCTCTGTGTACCAAACTTCAACAGCAAATCCGGCTCTTGGCAAGATTGGCAGATCAAAACTTTACGGGAAAGGTCTTAAGCAGGCAGGACGCAGTCTGGTCACTGAAACAGGAGGAAGACCTCTCTCATTTGTTCCACGAGCTGTCCTTGCCATGGATCCTCAGGCAGCCGAGAAATTTAGTCTTGACGGAAATATCGATTTACTGGTTGAAGTCACTTCTACAACTGTAAGAGAAGTAAATATCCAGATAGCTTATACAAGTGACACATTGTTCCTACACTGGGGTGCAATTCTTGACAACAAAGAAAATTGGGTTCTACCTTCTCGCTCTCCGGATAGAACTCAAAACTTCAAGAACAGTGCGCTTAGAACTCCATTTGTGAAATCCGGTGGCAATTCTCACCTTAAACTAGAGATAGATGATCCTGCCATACACGCTATTGAGTTCCTTATATTT

#### Obtention de la sequence muté

In [3]:
search_string = "SEX1[Gene] AND Arabidopsis thaliana[Organism]"
req = Entrez.esearch(db= "nucleotide", term=search_string)  # ID of fasta of SEX1 gene
res = Entrez.read(req)
res

DictElement({'Count': '5', 'RetMax': '5', 'RetStart': '0', 'IdList': ['1063682343', '1063682341', '334182442', '240254421', '332189094'], 'TranslationSet': [DictElement({'From': 'Arabidopsis thaliana[Organism]', 'To': '"Arabidopsis thaliana"[Organism]'}, attributes={})], 'TranslationStack': [DictElement({'Term': 'SEX1[Gene]', 'Field': 'Gene', 'Count': '8', 'Explode': 'N'}, attributes={}), DictElement({'Term': '"Arabidopsis thaliana"[Organism]', 'Field': 'Organism', 'Count': '2696552', 'Explode': 'Y'}, attributes={}), 'AND'], 'QueryTranslation': 'SEX1[Gene] AND "Arabidopsis thaliana"[Organism]'}, attributes={})

Ici on voit que les seules fasta disponible sont des variants d'epissage alternative, et vu que l'aligneur que nous allons construire ne va pas  être *splice-aware* , on aura des problèmes quand leurs introns n'aligneront pas avec notre sequence sauvage.

Importons alors, un version du SEX1 sauvage modifié afin de presenter 1/4 des SNP enregistré sur dbSNP pour ce gène:

In [5]:
import urllib.request  # the lib that handles the url stuff
from Bio.Seq import Seq
data = urllib.request.urlopen('https://raw.githubusercontent.com/SL-LAIDLAW/projet-outilBioinfo/dev/fasta/sex1_mutant_SNP.fasta') # it's a file like object and works just like a file
sex1_mutant = data.read()
sex1_mutant = sex1_mutant.decode('utf8').split('\n')[1:]
sex1_mutant = "".join(sex1_mutant)

sex1_mutant = Seq(sex1_mutant)
sex1_mutant

Seq('TATCTTCTGCAATTCGTGGTTTGTGAACACAAGAGAAAAAAAAAAAAAAACATA...AAG')

## Alignement des séquences associées aux gènes sauvages/mutés

### un alignement par paire global des séquences d’ARNm

Pour aligner nos sequences sauvages avec les mutants nous avons ecrit la fonction AlignSeqs qui effectue un alignement globale en utilisant l'algorithme Needleman–Wunsch.

In [6]:
!pip install cython
%load_ext Cython

/usr/local/lib/python2.7/site-packages/pip/_vendor/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [11]:
 %%cython
import pandas as pd

In [12]:
def InitializeTable(SeqX,SeqY):
  # intitialise dataframe holding scores
  df = pd.DataFrame(index=SeqY, columns=SeqX)
  col_names = list(df.keys())
  row_names = list(df.index)

  # initiate table
  df.iloc[0,0] = 0
  for i in range(len(SeqY)):
    df.iloc[i,0] = i * -1  # set row 0 to be -1,-2,-3
  for i in range(len(SeqX)):
    df.iloc[0,i] = i * -1  # set col 0 to be -1,-2,-3
  return df

In [13]:
%%cython
def PopulateTable(seqX,seqY,df):     
  df_dir = df.copy()
  # populate table
  for y in range(1,len(seqY)):
    for x in range(1,len(seqX)):
      current_score = -1000
      
      match_score = 1
      mismatch_score = -1

      if str(seqX[x]) == str(seqY[y]):
        current_score = match_score
      else:
        current_score = mismatch_score

      df.iloc[y,x] = current_score

      top_left_score = ((df.iloc[y-1,x-1]), '\x5C')
      top_score = ((df.iloc[y-1,x]), '\x7C')
      left_score = ((df.iloc[y,x-1]), '\x3C')

      best_score = top_score[0]
      best_direction = top_score[1]
      score_list = [top_left_score, top_score, left_score]
      for score_item in score_list:
        if int(score_item[0]) > int(best_score):
          best_direction = score_item[1]
          best_score = score_item[0]

        elif int(score_item[0]) == int(best_score):
          if not best_direction.__contains__(score_item[1]):
            best_direction += score_item[1]

      # delete score objects from memory as this caused issues
      for score in score_list:
        del score

      best_score = int(best_score)
      best_score += current_score

      df.iloc[y,x] = best_score
      df_dir.iloc[y,x] = best_direction
  return df, df_dir  

In [14]:
def Traceback(df,df_dir):
    # Start traceback
    my_alignment_x =[]
    my_alignment_y =[]
    bottom_right_y = list(df.shape)[0] -1
    bottom_right_x = list(df.shape)[1] -1

    x = bottom_right_x
    y = x - (bottom_right_x - bottom_right_y)
    while x != 0:
      current_coordinates = [y,x]
      top_coordinates = [y-1,x]
      left_coordinates = [y,x-1]
      top_left_coordinates = [y-1,x-1]

      current_nucleotide_x = df.keys()[x]
      current_nucleotide_y = df.index[y]
      current_direction = df_dir.iloc[y,x]

      if current_direction.__contains__('\\'):
        # move coordinates to top_left
        x = top_left_coordinates[1]
        y = top_left_coordinates[0]
        my_alignment_x.append(current_nucleotide_x)
        my_alignment_y.append(current_nucleotide_y)

      elif current_direction.__contains__('|'):
        # move coordinates to top
        x = top_coordinates[1]
        y = top_coordinates[0]
        my_alignment_x.append('-')
        my_alignment_y.append(current_nucleotide_y)

      elif current_direction.__contains__('<'):
        # move coordinates to left
        x = left_coordinates[1]
        y = left_coordinates[0]
        my_alignment_x.append(current_nucleotide_x)
        my_alignment_y.append('-')
        
      return my_alignment_x, my_alignment_y

In [15]:
def GetVariations(my_alignment_x, my_alignment_y):
  points_de_variations = set()
  for i in range(len(my_alignment_x)):
    x = my_alignment_x[i]
    y = my_alignment_y[i]

    if x != y:
      points_de_variations.add(i)
  return list(points_de_variations)

In [16]:
def AlignSeqs(SeqX,SeqY):
  SeqX = list(" " + SeqX)
  SeqY = list(" " + SeqY)

  match_score = 1
  mismatch_score = -1
  indel_score = -1
  
  penaltyArgs=[match_score, mismatch_score, indel_score]

  df = InitializeTable(SeqX,SeqY)

  df, df_dir = PopulateTableJit(SeqX,SeqY,df)


  my_alignment_x, my_alignment_y = Traceback(df,df_dir)

  my_alignment_x.reverse()
  my_alignment_y.reverse()

  points_de_variations = GetVariations(my_alignment_x, my_alignment_y)


  return my_alignment_x, my_alignment_y, points_de_variations

Cet étape prends beaucoup de temps donc est commenté pour l'instant

In [17]:
result_dict = {}
alignment_for_mutant, alignment_for_sauvage, variations = AlignSeqs(sex1_mutant, sex1_sauvage)
result_dict[mutant] = [alignment_for_mutant, alignment_for_sauvage, variations]

%store result_dict

KeyboardInterrupt: 

### donner les positions des variations sauvage/muté

In [ ]:
%store -r result_dict  # loads saved result from cache instead of re-running command

for key in result_dict.keys():
  result_list = result_dict[key]
  variants = result_list[2]
  for var in variants:
    print(var)

### taux de GC / traduction pour obtenir les séquences protéiques

In [18]:
from Bio.SeqUtils import GC
from Bio.Seq import Seq

taux_GC_sauvage = GC(sex1_sauvage)
print(taux_GC_sauvage)
taux_GC_mutant = GC(sex1_mutant)
print(taux_GC_mutant)

42.57936507936508
42.57820282924885


In [19]:
from Bio import SeqIO
from Bio import AlignIO
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord


trans_sauvage = sex1_sauvage.translate()
trans_mutant = sex1_mutant.translate()

print(trans_sauvage)
print(trans_mutant)

YLLQFVVCEHKRKKKKNIKKKEKLKKAKINGEKILYIIRKYHPNKKKRWSPQIPSINVSLGSLSPCRSSYVLE*RCATWWNRRIKCRFLLRLHVSVCMCE*VKKNKERRNPPALTSSIRSFKFWDLLSFSHCIQFLRLGSIIRSGKFDSVTRKHE*LCSA*LT*PGFDSSS*L*TSKQAQLLCVPNFNSKSGSWQDWQIKTLRERS*AGRTQSGH*NRRKTSLICSTSCPCHGSSGSREI*S*RKYRFTG*SHFYNCKRSKYPDSLYK*HIVPTLGCNS*QQRKLGSTFSLSG*NSKLQEQCA*NSICEIRWQFSP*TRDR*SCHTRY*VPYI*RKSEQMV*K*WSEFSYKLTNGKECETKCFCS*RSCTDPSIS*MGT*G*TNVQP*EREGGV*SRPDGATGGNDARCFSGRSQSKAVEER*Q**IPKI*WDIIQWTGGKEKSFQATRA*KKL*H*QDPAQGKGPD*AYL*ACC*FC*TRIQILI*TTVLNNSGDIRQSKGGTRNHSSL*QENIQA*RQCDFGVCY*TFRKDENSCGN*F*RAGYPSLGFVSKGWRMVGPTFRYTATKLFASTWCC*YKTDHHFNRSS*SGSNF*AGNRR*QLQGHAVCTQCW*KVD*K**Q*LLCGLC*RRKTCSEGLWRWKGYSQAFTGQNRRFGE*GPEVFHASIQHCSRSCGRGKKCWSTGLCRDPSLDEVYGYKTACVEQKL*C*AKGDKQSAG*TD*PSPGRLCKLSRVQRTFADDNVYCRSRR*RRCRATNP*RNSSHPAEK*LQGWNYGGMASEVA*QH*SR*CCHLSGIDGLYQK*L*LKCLLEDLER*WHNQRATLKL*SCYTF*TKF*RRTKRRSFA*SWTLHEDFKGCSFRGRP*VGYTKLHGLPR*R*RFHGWGADKSCIRIAFWISRLASFRPRTC*RKECRATS*GFA*SSSRAKATSAEVP*PPQGSVILGPRS*FYCQNSD*KRI*AIE*CWT*ENHVLHQPSS*KSCPLFR*Q*RPYILLE

# Base de données

## Reflexion sur le type de base de données

On a eu le choix d’un des SGBD vu ce semestre : chado/postgresql (module phenotype), relationnel seul (oracle, postgresqlou mysql), neo4j, couchdb, triplestore jena/rdf. 

On a choisi d'eviter les base de données NoSQL car nous avons plus d'experience avec ceux là. 

Notre choix s'est ainsi  porté sur la base de données Chado. Celle-ci est conçue spécialement pour gérer les représentations complexes de données biologiques. Chado va également posséder des modules permettant de gérer spécifiquement les relations entre termes ontologiques. Pour créer sa base de données Chado utilise le système  de gestion PostgreSQL. Ayant deja des bases en SQL , ce choix nous a ainsi paru judicieux  pour pouvoir réaliser une implémentation correcte de la base de données avec un temps limité et la taille de notre groupe.

## Modelisation de la base de données

Avant de nous lancer dans la création de notre base de donnée il faut d'abord la penser. Pour ce faire le meilleur moyen est de se représenter la base de données visuellement à l'aide d'un shéma UML et plus particulièrement d'un diagramme de classes. Notre bases de données se divise en "deux" parties. Une gérant les données issues des expérimentations des M1 BFP et l'autre gérant les termes d'ontologies. Un lien est fait entre ces 2 parties au niveau de la table phénotype. Avec le recul cette manière de faire n'était probablement pas la meilleure et des améliorations doivent être apportés principalement pour la partie gérant l'ontologie.


Nous retrouvons ansi avec une base de données corrspondant au diagramme de classe suivant:

![Diagramme de Classe](https://raw.githubusercontent.com/SL-LAIDLAW/projet-outilBioinfo/dev/img/class_diagram.png)



### Ontologie

Il a ainsi fallu penser à la manière d'intégrer l'ontologie dans notre base de données  . L'ontologie va représenter des concepts (comme une caractéristique phénotypique)   et les relations que ces concepts peuvent partager.  Dans notre cas on s'est intéressé à rechercher les relations entre les termes ontologiques constituant les phénotypes observés et mesurés par les étudiants BFP.  Pour ce faire ces phénotypes ont été "décomposé"  en plusieurs et leur terme ontologique ont été associé en une relation. Par exemple, la longueur de la racine primaire va corrrespondre à une longueur de racine (possédant son terme ontologique) qui ferait partie (is part of) d'une racine primaire(possédant aussi son propre terme ontologique).  ces données générées seront ensuite insérées dans notre base de données. Ces données peuvent être récupérées grâce à de nombreuses ressources regroupant les termes d'ontologies appartenant à un même "groupe". 

## Implantation de la base de données et peuplement

In [ ]:
from Bio import SeqIO
import pandas as pd

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/SL-LAIDLAW/projet-outilBioinfo/dev/CSV/Data_Projet_Entete.csv",sep=";")
df['Mutant'] = False
df['Mutant'].loc[df['Genotype'] != "Col_0"] = True
df.loc[df['Mutant'] == True]

## Création des tables

Afin de créer les tables un script SQL de création de ces tables à été écrit. Celui-ci va constituer dans un premier temps en la destruction des tables et de leurs contraintes dans le cas à l'aide d'un drop table. Il va ensuite permettre de construire ces tables en affectant les clefs primaires et étrangères appropriés à chaque table.

In [0]:
------Destruction des tables

drop table plant cascade ;
drop table phenotype cascade ;
drop table  boite cascade ;
drop table  milieu cascade ;
drop table  genotype cascade ;
drop table  gene cascade ;
drop table  cv cascade ;
drop table cvterm cascade ;
drop table  features cascade ;

--------------------------------------------------Creations tables--------------------------------------------

----table plant
create table plant (plant_id varchar(10) not null, primary key (plant_id), grouping int not null, plant_group int not null);

---table phenotype
create table phenotype (organism_id serial,primary key (organism_id), ShootArea real,  distFromHypocotyl real,  chlorophyll real, 
						Root_area real, tortuosity real, PR_lenght real, LR_lenght real,
						plant_id varchar(10) not null, constraint ph_fk foreign key (plant_id) references plant (plant_id) on delete cascade INITIALLY DEFERRED);

----table boite
create table boite (id_boite int, primary key(id_boite), plate varchar (20) not null, plant_id varchar (10), constraint b_fk foreign key (plant_id) references plant (plant_id) on delete cascade INITIALLY DEFERRED);

----table milieu
create table milieu(id_milieu int, primary key(id_milieu), type varchar (150) not null, 
			 id_boite int not null, constraint m_fk foreign key(id_boite) references boite(id_boite) on delete cascade INITIALLY DEFERRED);

----table genotype
create table gene(gene_id int not null, primary key(gene_id), gene_name varchar(20), est_mutant boolean);

----table gene
create table genotype(gene_id int not null, plant_id varchar(10), 
					constraint geno_pk primary key (gene_id, plant_id), 
					constraint gen_fk foreign key (gene_id) references gene(gene_id) on delete cascade INITIALLY DEFERRED, constraint plant_fk foreign key (plant_id) references plant(plant_id) on delete cascade INITIALLY DEFERRED);

---------------Tables Ontologies-----------

----table cv
create table cv(cv_id serial not null, primary key (cv_id), name varchar(255) not null, definition text, constraint cv_c1 unique (name));

----table cvterm
create table cvterm(cvterm_id serial not null,
    primary key (cvterm_id),
    cv_id int not null,
    constraint cvt_fk foreign key (cv_id) references cv (cv_id) on delete cascade INITIALLY DEFERRED ,
    name varchar(1024) not null,
    definition text);


create table features(feature_id serial not null,
    primary key (feature_id),  organism_id int not null,
    constraint f1_fk foreign key (organism_id) references phenotype (organism_id) on delete cascade INITIALLY DEFERRED,
    name varchar(20),
    uniquename text not null, type_id int not null,
    constraint f2_fk foreign key (type_id) references cvterm (cvterm_id) on delete cascade INITIALLY DEFERRED);

Afin de générer les tables nécessaires à l'ontologie, le module cv de Chado à été utilisé. Je me rends compte cependant à l'écriture de ce rapport que  j'ai mal compris et mal utilisé ce module cv, notamment en n'utilisant pas la table cv_term relationship.

## Insertions des données dans les tuples

Afin de ne pas avoir à insérer les données issues du document csv qui nous a été fourni manuellement dans nos tuples une réorganisation de celles-ci a été effectué. Celle-ci ont ainsi été redistribuées sur plusieurs fichier csv correspondant chacun à une table donnée. On peut ensuite insérer les données contenues dans ces fichier dans nos tuples à l'aide de la commande \copy . 

Ci-dessous le script de remplissage des tuples:

In [0]:
-----------------------CREATION DES TUPLES-------------------------------

----------------Plant-------------------------------

\copy plant FROM 'plant.csv' delimiter ',' CSV HEADER;

-----------------Phenotypes--------------------------

\copy phenotype FROM 'phenotype.csv' delimiter ',' CSV HEADER;

-----------------boite------------------------

\copy boite FROM 'boite.csv' delimiter ',' CSV HEADER;

-----------------Milieu-------------------------

\copy milieu FROM 'milieu.csv' delimiter ',' CSV HEADER;

-----------------gene-------------------------------

\copy gene FROM 'gene.csv' delimiter ',' CSV HEADER;


-----------------genotype------------------------

\copy genotype FROM 'genotype.csv' delimiter ',' CSV HEADER;

------------------cv--------------------------------

insert into cv values ( DEFAULT, 'http://purl.obolibrary.org/obo/pato.owl' , 'PATO');
insert into cv values ( DEFAULT, 'http://purl.obolibrary.org/obo/po.owl' , 'PO');
insert into cv values ( DEFAULT, 'http://purl.obolibrary.org/obo/flopo.owl', 'FLOPO');	
insert into cv values ( DEFAULT, 'http://purl.obolibrary.org/obo/to.owl', 'TO');
insert into cv values ( DEFAULT, 'Arabidopsis Phenotype Ontology','APO');

------------------cvterm----------------------------

insert into cvterm values (DEFAULT, 5, 'Root Lenght', 'a root lenght (FLOPO_0009325) which is part of a primary root (PO_0020127');
insert into cvterm values (DEFAULT, 5, 'Root Lenght', 'a root lenght (FLOPO_0009325) which is part of a lateral root (PO_0020121');
insert into cvterm values (DEFAULT, 5, 'area', 'an area (PATO_0001323 which is part of the shoot axis (PO_0025029)');
insert into cvterm values (DEFAULT, 5, 'area', 'an area (PATO_0001323) which is part of the root (PO_0009005)');
insert into cvterm values (DEFAULT, 5, 'distance', 'a distance(PATO_000040) between roots (PO_0009005) and hypocotyl (PO_0020100)');
insert into cvterm values (DEFAULT, 4, 'chlorophyll contents', 'Measures the chlorophyll content in a green tissue. Includes both chlorophyll-a and chlorophyll-b. Chlorophyll is the green pigment found in plants.');
insert into cvterm values (DEFAULT, 5, 'curvature', 'a curvature (PATO_0001591) which is part of roots (PO_0009005)');

------------------features--------------------------

insert into features values (DEFAULT, 1, 'PR_lenght', 'APO_000001', 1 );
insert into features values (DEFAULT, 1 , 'LR_lenght', 'APO_000002', 2);
insert into features values (DEFAULT, 1 ,'ShootArea', 'APO_000003',3);
insert into features values (DEFAULT, 1 , 'RootArea', 'APO_000004',4);
insert into features values (DEFAULT, 1 , 'DistanceFromHypocotyl', 'APO_000005',5);
insert into features values (DEFAULT, 1 , 'Chlorophylle','TO_0000495',6);
insert into features values (DEFAULT, 1 , 'Tortuosity', 'APO_000006', 7);

## Requêtes

Quelques requêtes ont ensuite été écrites afin de tester la base de données générée. Ces requêtes ont été regroupées dans le script ci-dessous. 

In [0]:
------Recupère l'id des plants ayant un génotype mutant et affiche la longueur de leur racines primaires et l'ontologie associée---------------- 

select ph.plant_id,ph.PR_lenght, f.name, f.uniquename,p.grouping, ct.definition 
from phenotype ph , features f, cvterm ct, genotype gt, gene g, plant p 
where g.gene_id = gt.gene_id and gt.plant_id = p.plant_id and p.plant_id= ph.plant_id and g.est_mutant = TRUE and f.name= 'PR_lenght' and f.type_id = ct.cvterm_id;



---------------Compare l'effet des mutations sur l'entrelacement des racines--------------

select g.gene_name, count(p.grouping)  
from gene g, plant p, genotype gt 
where g.gene_id = gt.gene_id and gt.plant_id = p.plant_id and p.grouping = 1 group by g.gene_name;




-----------Observe le potentiel effet du milieu sur la taille moyenne de l'aire racinaire--------- 

select m.type, avg(ph.Root_area)
from milieu m, boite b, plant p, phenotype ph
where m.id_boite = b.id_boite and b.plant_id = p.plant_id and p.plant_id = ph.plant_id 
group by m.type;

-----------------Affiche l'id de tout les individus sauf ceux Col 0-------------------------

select p.plant_id from plant p, genotype gt,gene g
where g.gene_id = gt.gene_id and gt.plant_id = p.plant_id
EXCEPT
select gt.plant_id from plant p, gene g , genotype gt
where g.gene_id = gt.gene_id and gt.plant_id = p.plant_id and g.gene_name = 'Col_0';

----------------Affiche 

select f.name, f.uniquename,ct.name, ct.definition  from features f, cvterm ct, cv c
where c.cv_id = ct.cv_id and ct.cvterm_id = f.type_id and c.definition ='APO';

Ces  requêtes m'ont permis de me rendre compte que certains éléments de la base de donnée auraient mérités à être pensé autrement afin de pouvoir faire des requêtes plus précise et intéressantes au niveau de l'ontologie.

# Conclusion


Ainsi nous avons été en mesure de générer une base de donnée permettant de faire des requêtes pour la consultation et potentiellement l'interprétation des résultats obtenus par les M1 BFP . Cette base de données permet aussi de consulter les relations d'ontologies des phénotypes mesurés. Cependant cette ontologie n'a pas été implémenté de manière optimale par manque de comprhéension et de temps pour rattraper les erreurs. Les séquences nucléiques des gènes o,t pu être récupéreés depuis le NCBI puis elles ont pu être analysées àl'aide de BioPython pour nous fournir des informations qui auraient pu être utilisées dans notre base de données. 